In [20]:
import tensorflow as tf

def dice_loss(y_true, y_pred):
    smooth = 1e-15
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    score = (2. * intersection + smooth) / (union + smooth)
    return 1. - score

def dice_coef(y_true, y_pred):
    smooth = 1e-15
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    score = (2. * intersection + smooth) / (union + smooth)
    return score

In [21]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [22]:
H = 256
W = 256


In [23]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [24]:
def load_dataset(image_path, mask_path, split = 0.2):
    images = sorted(glob(os.path.join(image_path, "images", "*.png")))
    masks = sorted(glob(os.path.join(mask_path, "masks", "*.png")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size = split_size, random_state = 42)
    train_y, valid_y = train_test_split(masks, test_size = split_size, random_state = 42)
    train_x, test_x = train_test_split(train_x, test_size = split_size, random_state = 42)
    train_y, test_y = train_test_split(train_y, test_size = split_size, random_state = 42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [25]:
def read_image(path):
    x = cv2.imread(path.numpy().decode(), cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x


In [26]:
def read_mask(path):
    x = cv2.imread(path.numpy().decode(), cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x


In [27]:
def tf_parse(image_path, mask_path):
    def _parse(image_path, mask_path):

        image = read_image(image_path)
        mask = read_mask(mask_path)

        return image, mask

    image, mask = tf.py_function(_parse, [image_path, mask_path], [tf.float32, tf.float32])
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, 1])
    return image, mask


In [28]:
def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


In [29]:
if __name__ == "__main__":

    tf.config.run_functions_eagerly(True)

    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir("/kaggle/working/files")

    batch_size = 5
    lr = 1e-4
    num_epoch = 25
    model_path = "/kaggle/working/files/model.keras"
    csv_path = "/kaggle/working/files/log.csv"



In [30]:
    image_path = "/kaggle/input/fimages"
    mask_path = "/kaggle/input/fmasks1"

In [31]:
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(image_path, mask_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 1862 - 1862
Valid: 620 - 620
Test: 620 - 620


In [32]:
    train_dataset = tf_dataset(train_x, train_y, batch = batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch = batch_size)

/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [33]:
print("Shapes in train_dataset:")
for image_batch, mask_batch in train_dataset:
  print("Image batch shape:", image_batch.shape)
  print("Mask batch shape:", mask_batch.shape)


Shapes in train_dataset:
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape

In [34]:
print("Shapes in valid_dataset:")
for image_batch, mask_batch in valid_dataset:
  print("Image batch shape:", image_batch.shape)
  print("Mask batch shape:", mask_batch.shape)


Shapes in valid_dataset:
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape: (5, 256, 256, 1)
Image batch shape: (5, 256, 256, 3)
Mask batch shape

In [35]:
train_dataset = train_dataset.filter(lambda x, y: tf.math.reduce_any(tf.math.logical_not(tf.math.equal(y, 0))))
valid_dataset = valid_dataset.filter(lambda x, y: tf.math.reduce_any(tf.math.logical_not(tf.math.equal(y, 0))))


In [36]:
print("Cardinality of train_dataset:", train_dataset.cardinality())
print("Cardinality of valid_dataset:", valid_dataset.cardinality())

Cardinality of train_dataset: tf.Tensor(-2, shape=(), dtype=int64)
Cardinality of valid_dataset: tf.Tensor(-2, shape=(), dtype=int64)


In [37]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [38]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # print(s1.shape, s2.shape, s3.shape, s4.shape)
    # print(p1.shape, p2.shape, p3.shape, p4.shape)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model


In [39]:
if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_unet(input_shape)
    model.summary()


Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 256, 256,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 256, 256,  │     36,928 │ activation_18[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_20[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 128, 128,  │          0 │ activation_19[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 128, 128,  │     73,856 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 128, 128,  │    147,584 │ activation_20[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_21       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 64, 64,    │          0 │ activation_21[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 64, 64,    │    295,168 │ max_pooling2d_5[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_23[0][0] 

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [40]:
model = build_unet((W, H, 3))
model.compile(loss = dice_loss, optimizer = Adam(lr), metrics = [dice_coef])

In [41]:
callbacks = [
    ModelCheckpoint(model_path, verbose = 1, save_best_only = True),
    ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 5, min_lr = 1e-4, verbose = 1),
    CSVLogger(csv_path),
    EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = False)
    ]

In [42]:
model.fit(
    train_dataset,
    epochs=num_epoch,
    validation_data=valid_dataset,
    callbacks=callbacks
    )

Epoch 1/25
    373/Unknown 324s 808ms/step - dice_coef: 0.1500 - loss: 0.8500

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 0.93298, saving model to /kaggle/working/files/model.keras
373/373 ━━━━━━━━━━━━━━━━━━━━ 343s 861ms/step - dice_coef: 0.1502 - loss: 0.8498 - val_dice_coef: 0.0670 - val_loss: 0.9330 - learning_rate: 1.0000e-04
Epoch 2/25
373/373 ━━━━━━━━━━━━━━━━━━━━ 0s 785ms/step - dice_coef: 0.3861 - loss: 0.6139
Epoch 2: val_loss improved from 0.93298 to 0.75576, saving model to /kaggle/working/files/model.keras
373/373 ━━━━━━━━━━━━━━━━━━━━ 312s 837ms/step - dice_coef: 0.3862 - loss: 0.6138 - val_dice_coef: 0.2442 - val_loss: 0.7558 - learning_rate: 1.0000e-04
Epoch 3/25
373/373 ━━━━━━━━━━━━━━━━━━━━ 0s 786ms/step - dice_coef: 0.5037 - loss: 0.4963
Epoch 3: val_loss improved from 0.75576 to 0.59486, saving model to /kaggle/working/files/model.keras
373/373 ━━━━━━━━━━━━━━━━━━━━ 323s 839ms/step - dice_coef: 0.5038 - loss: 0.4962 - val_dice_coef: 0.4051 - val_loss: 0.5949 - learning_rate: 1.0000e-04
Epoch 4/25
373/373 ━━━━━━━━━━━━━━━━━━━━ 0s 784ms/step - dice_coef:

In [43]:
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
import numpy as np
import cv2


In [44]:
def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    mask = mask.astype(np.uint8)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred.astype(np.uint8) * 255

    H, W, _ = image.shape

    line = np.ones((H, 10, 3), dtype=np.uint8) * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [45]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("/kaggle/working/results")

In [46]:

    with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
        model = tf.keras.models.load_model(os.path.join("/kaggle/working/files", "model.keras"))

In [47]:
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total = len(test_y)):
  name = x.split("/")[-1]
  image = cv2.imread(x, cv2.IMREAD_COLOR)
  image = cv2.resize(image, (W, H))
  x = image / 255.0
  x = x.astype(np.float32)

  mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
  mask = cv2.resize(mask, (W, H))

  x = x.reshape(-1, 256, 256, 3)
  y_pred = model.predict(x, verbose = 0)[0]
  y_pred = np.squeeze(y_pred, axis = -1)
  y_pred = y_pred >= 0.5
  y_pred = y_pred.astype(np.int32)

  result_path = "/content/drive/MyDrive/Project/results"
  save_image_path = os.path.join(result_path, name)
  save_results(image, mask, y_pred, save_image_path)

  mask = mask/255.0
  mask = (mask > 0.5).astype(np.int32).flatten()
  y_pred = y_pred.flatten()

  f1_value = f1_score(mask, y_pred, labels = [0, 1], average = "binary")
  jac_value = accuracy_score(mask, y_pred)
  recall_value = recall_score(mask, y_pred, labels = [0, 1], average = "binary")
  precision_value = precision_score(mask, y_pred, labels = [0, 1], average = "binary")
  SCORE.append([name, f1_value, jac_value, recall_value, precision_value])

  0%|          | 1/620 [00:03<33:42,  3.27s/it]/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
  0%|          | 2/620 [00:03<15:13,  1.48s/it]/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
  0%|          | 3/620 [00:03<09:19,  1.10it/s]/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run

In [48]:
score = [s[1:] for s in SCORE]
score = np.mean(score, axis = 0)
print(f"F1: {score[0]:0.5f}")
print(f"Accuracy: {score[1]:0.5f}")
print(f"Recall: {score[2]:0.5f}")
print(f"Precision: {score[3]:0.5f}")

F1: 0.71835
Accuracy: 0.99278
Recall: 0.69801
Precision: 0.81108


In [49]:
df = pd.DataFrame(SCORE, columns = ["Image", "F1", "Accuracy", "Recall", "Precision"])
df.to_csv("/kaggle/working/files/score.csv")